In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StringType, IntegerType, StructType, DoubleType, DecimalType
from pyspark.sql.functions import when, lit, col

In [2]:
spark = SparkSession.builder.appName("ETL").getOrCreate()

raw_data_loc = "../data/raw/application/application_train.csv"
df_raw = spark.read.csv(raw_data_loc, inferSchema=True, header=True)

cleaned_data_loc = "../data/processed/app_train_cleaned/"

In [3]:
#df_raw.show()
# df_raw.columns
df_raw.printSchema()
# df_raw.describe()

root
 |-- SK_ID_CURR: integer (nullable = true)
 |-- TARGET: integer (nullable = true)
 |-- NAME_CONTRACT_TYPE: string (nullable = true)
 |-- CODE_GENDER: string (nullable = true)
 |-- FLAG_OWN_CAR: string (nullable = true)
 |-- FLAG_OWN_REALTY: string (nullable = true)
 |-- CNT_CHILDREN: integer (nullable = true)
 |-- AMT_INCOME_TOTAL: double (nullable = true)
 |-- AMT_CREDIT: double (nullable = true)
 |-- AMT_ANNUITY: double (nullable = true)
 |-- AMT_GOODS_PRICE: double (nullable = true)
 |-- NAME_TYPE_SUITE: string (nullable = true)
 |-- NAME_INCOME_TYPE: string (nullable = true)
 |-- NAME_EDUCATION_TYPE: string (nullable = true)
 |-- NAME_FAMILY_STATUS: string (nullable = true)
 |-- NAME_HOUSING_TYPE: string (nullable = true)
 |-- REGION_POPULATION_RELATIVE: double (nullable = true)
 |-- DAYS_BIRTH: integer (nullable = true)
 |-- DAYS_EMPLOYED: integer (nullable = true)
 |-- DAYS_REGISTRATION: double (nullable = true)
 |-- DAYS_ID_PUBLISH: integer (nullable = true)
 |-- OWN_CAR_AG

In [4]:
# val dfNew = df.withColumn("newColName", df.originalColName.cast(IntegerType))
#     .drop("originalColName").withColumnRenamed("newColName", "originalColName")

In [15]:
# Replace DAYS_BIRTH with AGE
df_age = df_raw.withColumn("AGE", (df_raw['DAYS_BIRTH'] / -365).cast(IntegerType())).drop("DAYS_BIRTH")
# df_age.select("AGE").show()
# df_age.select("DAYS_BIRTH").show()

# Replace DAYS_EMPLOYED with YEARS_EMPLOYED
df_employed = df_age.withColumn("YEARS_EMPLOYED_with_anom", (df_age["DAYS_EMPLOYED"] / -365).cast(DecimalType(10,4))).drop("DAYS_EMPLOYED")
# df_employed.select("YEARS_EMPLOYED").show()

# anom_YEARS_EMPLOYED
df_employed_anom = df_employed.withColumn("YEARS_EMPLOYED_anom", (df_employed["YEARS_EMPLOYED_with_anom"] < 0).cast(IntegerType()))
df_employed_anom = df_employed_anom.withColumn("YEARS_EMPLOYED", when(df_employed["YEARS_EMPLOYED_with_anom"] >= 0, df_employed["YEARS_EMPLOYED_with_anom"]))
df_employed_anom = df_employed_anom.drop("YEARS_EMPLOYED_with_anom")
# df_employed_anom.select(["YEARS_EMPLOYED_anom", "YEARS_EMPLOYED"]).show()

def replace_with_NA(col, to_replace):
    return when(col != to_replace, col)

# Remove the 2 Unknown values
df_fam_null = df_employed_anom.withColumn("NAME_FAMILY_STATUS", 
                                            replace_with_NA(df_employed_anom["NAME_FAMILY_STATUS"],
                                                            "Unknown"
                                                           )
                                         )
# df_fam_null.select("NAME_FAMILY_STATUS").filter(col("NAME_FAMILY_STATUS").isNull()).show()

# Remove the 4 XNA values
df_gender_null = df_fam_null.withColumn("CODE_GENDER", 
                                            replace_with_NA(df_employed_anom["CODE_GENDER"],
                                                            "XNA"
                                                           )
                                       )
# df_gender_null.select("CODE_GENDER").filter(col("CODE_GENDER").isNull()).show()
# df_gender_null.select("CODE_GENDER").show()

df_gender_null.count() == df_raw.count()

True

In [14]:
# Export
df_gender_null.write.mode("Overwrite").csv(cleaned_data_loc)
df_gender_null.toPandas().to_csv(cleaned_data_loc + "app_train_processed.csv", index=False)